In [3]:
from platform import python_version
print(python_version())

3.8.7


In [4]:
#https://crime-data-explorer.app.cloud.gov/explorer/state/georgia/crime

In [47]:
import pandas as pd
import numpy as np, sys, time, pdb, matplotlib.pyplot as plt
from pymongo import MongoClient

In [48]:
pd.set_option('display.max_rows', 100)# None

In [49]:
data_2020 = pd.read_csv('./flask/static/data/COBRA-2020.csv')
data_2021 = pd.read_csv('./flask/static/data/COBRA-2021.csv')
data_2009_2019 = pd.read_csv('./flask/static/data/COBRA-2009-2019.csv', low_memory=False)

In [ ]:
data_2009_2019.describe()

In [ ]:
data_2020_list = data_2020.columns.values.tolist()
data_2021_list = data_2021.columns.values.tolist()
data_2009_2019_list = data_2009_2019.columns.values.tolist()

In [ ]:
print(len(data_2020_list))
data_2020.head(1)

In [ ]:
print(len(data_2009_2019_list))
data_2009_2019.tail(10)

In [ ]:
#Since the 2009-2019 dataset is the largest, everything will try and have the same columns as that dataset
data_2020.rename(columns = {
 'offense_id' : 'Report Number',
 'rpt_date' : 'Report Date',
 'occur_date' : 'Occur Date',
 'occur_time' : 'Occur Time',
 'poss_date' : 'Possible Date',
 'poss_time' : 'Possible Time',
 'beat' : 'Beat',
 'apt_office_prefix' : 'Apartment Office Prefix',
 'apt_office_num' : 'Apartment Number',
 'location' : 'Location',
 #'MinOfucr' : ,
 #'dispo_code' : ,
 'Shift' : 'Shift Occurence',
 'loc_type' : 'Location Type',
 'UC2_Literal': 'UCR Literal',
 'ibr_code' : 'IBR Code',
 'neighborhood' : 'Neighborhood',
 'npu' : 'NPU',
 'long' : 'Longitude',
 'lat' : 'Latitude'
}, inplace = True)
#Drop columns that do not seem to show up in 2009-2019 dataset
data_2020 = data_2020.drop(columns=['MinOfucr','dispo_code'])
data_2020.head(1)

In [ ]:
list1 = data_2009_2019.columns.values.tolist()
list2 = data_2020.columns.values.tolist()

list_difference = [item for item in list1 if item not in list2]
print(list_difference)

In [ ]:
#It can be seen that the 'UCR #' shows up only in the 2009-2019 data and realistically serves no purpose because its a code only a cop would udnerstand
#So it should be droped to simplify
data_2009_2019 = data_2009_2019.drop(columns=['UCR #'])

In [ ]:
list1 = data_2009_2019.columns.values.tolist()
list2 = data_2020.columns.values.tolist()

list_difference = [item for item in list1 if item not in list2]
print(list_difference)
#There should now be no differences as at all

In [ ]:
#Concat the dataframes together
data = pd.concat([data_2009_2019, data_2020])

In [ ]:
data_2009_2019['Occur Date'].sort_values()

In [ ]:
data['Report Date'] = pd.to_datetime(data['Report Date'])
#data['Occur Date'] = pd.to_datetime(data['Occur Date'])

In [ ]:
#data = data.astype({'Report Date':'datetime64','Occur Date':'datetime64','Possible Date':'datetime64'},errors = 'raise')

In [ ]:
data.dtypes

In [ ]:
data.describe

In [ ]:
data.to_csv('data.csv')

In [ ]:
df.reset_index(drop=True, inplace=True)
df.to_json('data.json')

# Machine Learnings

## Read the CSV and Perform Basic Data Cleaning

In [ ]:
df = pd.read_csv("data.csv")
# Drop the null columns where all values are null
#df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df[["Latitude","Longitude","Neighborhood"]]
df = df.dropna()

## Create a Train Test Split

Use `Neighborhood` for the y values
This can be an idiot check to see if lat an lon can predict neighborhood

In [ ]:
from sklearn.model_selection import train_test_split
y = df["Neighborhood"]
X = df.drop(columns=["Neighborhood"])

#For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)#Compute the minimum and maximum to be used for later scaling.

X_train_scaled = X_scaler.transform(X_train)#Scale features of X_train according to feature_range.
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
y_train.nunique()

# Train the Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(max_iter=200)
model.fit(X_train_scaled, y_train)# Fit the LR model

In [ ]:
df

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

In [ ]:
model.predict()

In [ ]:
X_scaler.transform(X_train)[0]

In [ ]:
model.predict(X_train_scaled)

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
# Cfloat, default=1.0 Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
# penalty{‘l1’, ‘l2’, ‘elasticnet’, ‘none’}, default=’l2’
param_grid = {'C': [1, 3,5,7,9],
              'penalty': ["l1", "l2", "none"]}
Model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(Model, param_grid, verbose=3)
grid

In [ ]:
idk = grid.fit(X_train_scaled, y_train)
idk

# Need to Decrease Sample Size

It has become apparent that there is too mach data and our relatively weak computers are not up to the challenge give the time.

It was decided to just look at 5 major "Neighborhood" of interest:

buckhead, midtown, brookhaven, downtown, old fourth ward

In [23]:
df = pd.read_csv("data.csv")
# Drop the null columns where all values are null
#df = df.dropna(axis='columns', how='all')
# Drop the null rows
#df_reduced = df[["Latitude","Longitude","Neighborhood"]]
#df_reduced = df_reduced.dropna()
df_reduced = df

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (4,6,7,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
df_reduced["Neighborhood"].unique();

In [26]:
#Used to show all rows for error checking
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 100)

In [27]:
df_buckhead = df_reduced[(df_reduced["Neighborhood"] == "Buckhead Forest") |(df_reduced["Neighborhood"] == "Buckhead Village")
           |(df_reduced["Neighborhood"] == "North Buckhead") |(df_reduced["Neighborhood"] == "Buckhead Heights")
          ]
df_buckhead

,Unnamed: 0,Report Number,Report Date,Occur Date,Occur Time,Possible Date,Possible Time,Beat,Apartment Office Prefix,Apartment Number,Location,Shift Occurence,Location Type,UCR Literal,IBR Code,Neighborhood,NPU,Latitude,Longitude
13,13,90010745,2009-01-01,2008-12-31,2030,2009-01-01,800.0,208.0,NaN,313,3558 PIEDMONT RD NE,Day Watch,18,LARCENY-FROM VEHICLE,2305,Buckhead Forest,B,33.851150,-84.379440
76,76,90010159,2009-01-01,2009-01-01,57,2009-01-01,105.0,206.0,NaN,NaN,BUCKEAD AVE/GRANDVIEW AVE,Morning Watch,99,ROBBERY-PEDESTRIAN,1204,Buckhead Village,B,33.837520,-84.375740
118,118,90021591,2009-01-02,2009-01-02,2110,2009-01-02,2130.0,208.0,NaN,86,86 ROSWELL CT NE,Unknown,26,BURGLARY-RESIDENCE,2204,Buckhead Forest,B,33.848830,-84.381740
217,217,90020096,2009-01-02,2009-01-01,2215,2009-01-02,130.0,208.0,NaN,NaN,3365 PIEDMONT RD NE,Morning Watch,18,LARCENY-FROM VEHICLE,2305,North Buckhead,B,33.847090,-84.372950
226,226,90020730,2009-01-02,2009-01-02,148,2009-01-02,150.0,208.0,NaN,NaN,3398 PIEDMONT RD NE,Day Watch,14,LARCENY-FROM VEHICLE,2305,Buckhead Forest,B,33.847120,-84.373850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350055,7141,202751750,2020-10-01,10/1/2020,17:59,10/1/2020,18:50,208,NaN,NaN,3500 PEACHTREE RD NE,NaN,NaN,LARCENY-FROM VEHICLE,23F,North Buckhead,B,33.852786,-84.362043
350059,7145,202751862,2020-10-01,9/29/2020,16:00,10/1/2020,21:00,208,NaN,NaN,3566 PIEDMONT RD NE,NaN,NaN,AUTO THEFT,240,Buckhead Forest,B,33.851598,-84.380612
350082,7168,202741807,2020-10-01,9/30/2020,19:30,9/30/2020,22:15,206,NaN,NaN,334 EAST PACES FERRY RD NE,NaN,NaN,LARCENY-NON VEHICLE,23H,Buckhead Village,B,33.839330,-84.376588
350127,7213,202741590,2020-09-30,9/30/2020,19:20,9/30/2020,19:44,206,NaN,NaN,3221 PEACHTREE RD,NaN,NaN,LARCENY-FROM VEHICLE,23F,Buckhead Village,B,33.842061,-84.373026


In [28]:
df_buckhead['Neighborhood']

13         Buckhead Forest
76        Buckhead Village
118        Buckhead Forest
217         North Buckhead
226        Buckhead Forest
                ...       
350055      North Buckhead
350059     Buckhead Forest
350082    Buckhead Village
350127    Buckhead Village
350129    Buckhead Village
Name: Neighborhood, Length: 10156, dtype: object

In [29]:
df_buckhead = df_buckhead.assign(Neighborhood = 'Buckhead' )
df_buckhead

,Unnamed: 0,Report Number,Report Date,Occur Date,Occur Time,Possible Date,Possible Time,Beat,Apartment Office Prefix,Apartment Number,Location,Shift Occurence,Location Type,UCR Literal,IBR Code,Neighborhood,NPU,Latitude,Longitude
13,13,90010745,2009-01-01,2008-12-31,2030,2009-01-01,800.0,208.0,NaN,313,3558 PIEDMONT RD NE,Day Watch,18,LARCENY-FROM VEHICLE,2305,Buckhead,B,33.851150,-84.379440
76,76,90010159,2009-01-01,2009-01-01,57,2009-01-01,105.0,206.0,NaN,NaN,BUCKEAD AVE/GRANDVIEW AVE,Morning Watch,99,ROBBERY-PEDESTRIAN,1204,Buckhead,B,33.837520,-84.375740
118,118,90021591,2009-01-02,2009-01-02,2110,2009-01-02,2130.0,208.0,NaN,86,86 ROSWELL CT NE,Unknown,26,BURGLARY-RESIDENCE,2204,Buckhead,B,33.848830,-84.381740
217,217,90020096,2009-01-02,2009-01-01,2215,2009-01-02,130.0,208.0,NaN,NaN,3365 PIEDMONT RD NE,Morning Watch,18,LARCENY-FROM VEHICLE,2305,Buckhead,B,33.847090,-84.372950
226,226,90020730,2009-01-02,2009-01-02,148,2009-01-02,150.0,208.0,NaN,NaN,3398 PIEDMONT RD NE,Day Watch,14,LARCENY-FROM VEHICLE,2305,Buckhead,B,33.847120,-84.373850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350055,7141,202751750,2020-10-01,10/1/2020,17:59,10/1/2020,18:50,208,NaN,NaN,3500 PEACHTREE RD NE,NaN,NaN,LARCENY-FROM VEHICLE,23F,Buckhead,B,33.852786,-84.362043
350059,7145,202751862,2020-10-01,9/29/2020,16:00,10/1/2020,21:00,208,NaN,NaN,3566 PIEDMONT RD NE,NaN,NaN,AUTO THEFT,240,Buckhead,B,33.851598,-84.380612
350082,7168,202741807,2020-10-01,9/30/2020,19:30,9/30/2020,22:15,206,NaN,NaN,334 EAST PACES FERRY RD NE,NaN,NaN,LARCENY-NON VEHICLE,23H,Buckhead,B,33.839330,-84.376588
350127,7213,202741590,2020-09-30,9/30/2020,19:20,9/30/2020,19:44,206,NaN,NaN,3221 PEACHTREE RD,NaN,NaN,LARCENY-FROM VEHICLE,23F,Buckhead,B,33.842061,-84.373026


In [30]:
df_other_hoods  = df_reduced[(df_reduced["Neighborhood"] == "Midtown") | (df_reduced["Neighborhood"] == "Brookhaven")
          |(df_reduced["Neighborhood"] == "Downtown") | (df_reduced["Neighborhood"] == "Old Fourth Ward")]
df_other_hoods

,Unnamed: 0,Report Number,Report Date,Occur Date,Occur Time,Possible Date,Possible Time,Beat,Apartment Office Prefix,Apartment Number,Location,Shift Occurence,Location Type,UCR Literal,IBR Code,Neighborhood,NPU,Latitude,Longitude
1,1,90011083,2009-01-01,2009-01-01,1330,2009-01-01,1330.0,511.0,NaN,NaN,12 BROAD ST SW,Day Watch,9,LARCENY-NON VEHICLE,2303,Downtown,M,33.753200,-84.392010
9,9,90010108,2009-01-01,2008-12-31,2000,2009-01-01,100.0,507.0,NaN,NaN,360 MARIETTA ST NW,Morning Watch,18,LARCENY-FROM VEHICLE,2305,Downtown,M,33.762690,-84.397020
15,15,90010805,2009-01-01,2009-01-01,900,2009-01-01,945.0,604.0,NaN,1305,626 DEKALB AVE SE,Day Watch,18,LARCENY-FROM VEHICLE,2305,Old Fourth Ward,M,33.752830,-84.366870
16,16,90010917,2009-01-01,2008-12-31,1915,2009-01-01,1137.0,503.0,NaN,NaN,422 9TH ST NE,Day Watch,13,LARCENY-FROM VEHICLE,2305,Midtown,E,33.780800,-84.373340
17,17,90011020,2009-01-01,2008-12-31,1900,2009-01-01,1230.0,509.0,NaN,NaN,590 W PEACHTREE ST NW,Day Watch,18,LARCENY-FROM VEHICLE,2305,Downtown,M,33.770230,-84.388150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350126,7212,202741548,2020-09-30,2/16/2020,10:00,3/9/2020,18:00,502,NaN,NaN,1045 PIEDMONT AVE NE,NaN,NaN,LARCENY-NON VEHICLE,23H,Midtown,E,33.783060,-84.379421
350138,7224,202740394,2020-09-30,9/29/2020,18:00,9/30/2020,6:00,508,NaN,NaN,344 LUCKIE ST NW,NaN,NaN,BURGLARY,220,Downtown,M,33.763927,-84.396356
350145,7231,202740612,2020-09-30,9/28/2020,22:00,9/29/2020,8:00,603,NaN,NaN,525 GLEN IRIS DR NE,NaN,NaN,LARCENY-FROM VEHICLE,23F,Old Fourth Ward,M,33.768814,-84.366527
350156,7242,202740728,2020-09-30,9/26/2020,22:34,9/27/2020,12:00,503,NaN,NaN,1138 PEACHTREE ST NE,NaN,NaN,LARCENY-FROM VEHICLE,23F,Midtown,E,33.785508,-84.383434


In [31]:
df_5hoods = pd.concat([df_buckhead,df_other_hoods], ignore_index=True, sort=False)
df_5hoods

,Unnamed: 0,Report Number,Report Date,Occur Date,Occur Time,Possible Date,Possible Time,Beat,Apartment Office Prefix,Apartment Number,Location,Shift Occurence,Location Type,UCR Literal,IBR Code,Neighborhood,NPU,Latitude,Longitude
0,13,90010745,2009-01-01,2008-12-31,2030,2009-01-01,800.0,208.0,NaN,313,3558 PIEDMONT RD NE,Day Watch,18,LARCENY-FROM VEHICLE,2305,Buckhead,B,33.851150,-84.379440
1,76,90010159,2009-01-01,2009-01-01,57,2009-01-01,105.0,206.0,NaN,NaN,BUCKEAD AVE/GRANDVIEW AVE,Morning Watch,99,ROBBERY-PEDESTRIAN,1204,Buckhead,B,33.837520,-84.375740
2,118,90021591,2009-01-02,2009-01-02,2110,2009-01-02,2130.0,208.0,NaN,86,86 ROSWELL CT NE,Unknown,26,BURGLARY-RESIDENCE,2204,Buckhead,B,33.848830,-84.381740
3,217,90020096,2009-01-02,2009-01-01,2215,2009-01-02,130.0,208.0,NaN,NaN,3365 PIEDMONT RD NE,Morning Watch,18,LARCENY-FROM VEHICLE,2305,Buckhead,B,33.847090,-84.372950
4,226,90020730,2009-01-02,2009-01-02,148,2009-01-02,150.0,208.0,NaN,NaN,3398 PIEDMONT RD NE,Day Watch,14,LARCENY-FROM VEHICLE,2305,Buckhead,B,33.847120,-84.373850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65538,7212,202741548,2020-09-30,2/16/2020,10:00,3/9/2020,18:00,502,NaN,NaN,1045 PIEDMONT AVE NE,NaN,NaN,LARCENY-NON VEHICLE,23H,Midtown,E,33.783060,-84.379421
65539,7224,202740394,2020-09-30,9/29/2020,18:00,9/30/2020,6:00,508,NaN,NaN,344 LUCKIE ST NW,NaN,NaN,BURGLARY,220,Downtown,M,33.763927,-84.396356
65540,7231,202740612,2020-09-30,9/28/2020,22:00,9/29/2020,8:00,603,NaN,NaN,525 GLEN IRIS DR NE,NaN,NaN,LARCENY-FROM VEHICLE,23F,Old Fourth Ward,M,33.768814,-84.366527
65541,7242,202740728,2020-09-30,9/26/2020,22:34,9/27/2020,12:00,503,NaN,NaN,1138 PEACHTREE ST NE,NaN,NaN,LARCENY-FROM VEHICLE,23F,Midtown,E,33.785508,-84.383434


In [92]:
df_5hoods = df_5hoods.drop(columns=['Unnamed: 0'])
df_5hoods.to_csv('data_5hoods.csv', index= False)

KeyError: "['Unnamed: 0'] not found in axis"

# Machine Learnings

In [139]:
df = pd.read_csv("data_5hoods.csv")
#df = df[["Latitude","Longitude","Neighborhood"]]
#

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (7,8,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [140]:
df.head(1)

,Report Number,Report Date,Occur Date,Occur Time,Possible Date,Possible Time,Beat,Apartment Office Prefix,Apartment Number,Location,Shift Occurence,Location Type,UCR Literal,IBR Code,Neighborhood,NPU,Latitude,Longitude
0,90010745,2009-01-01,2008-12-31,2030,2009-01-01,800.0,208.0,NaN,313,3558 PIEDMONT RD NE,Day Watch,18,LARCENY-FROM VEHICLE,2305,Buckhead,B,33.85115,-84.37944


In [141]:
df = df.drop(columns=['Apartment Office Prefix',
                      'Report Date',
                      'Occur Date',
                      'Occur Time',
                      'Possible Time',
                      'Apartment Number',
                      'Location',
                      'Shift Occurence',
                      'UCR Literal',
                      'NPU',
                      'Possible Date',
                      'IBR Code',
                      'Location Type'
                     ])

In [142]:
pd.set_option('display.max_rows', 1000)# None
df = df.dropna()
df[pd.to_numeric(df['Beat'], errors='coerce').notnull()]
#df[pd.to_numeric(df['Location Type'], errors='coerce').notnull()]
print(df.dtypes)
df.head(1)

Report Number      int64
Beat             float64
Neighborhood      object
Latitude         float64
Longitude        float64
dtype: object


,Report Number,Beat,Neighborhood,Latitude,Longitude
0,90010745,208.0,Buckhead,33.85115,-84.37944


In [143]:
from sklearn.model_selection import train_test_split
y = df["Neighborhood"]
X = df.drop(columns=["Neighborhood"])
#For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [144]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)#Compute the minimum and maximum to be used for later scaling.

X_train_scaled = X_scaler.transform(X_train)#Scale features of X_train according to feature_range.
X_test_scaled = X_scaler.transform(X_test)

In [145]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
# Cfloat, default=1.0 Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
# penalty{‘l1’, ‘l2’, ‘elasticnet’, ‘none’}, default=’l2’
param_grid = {'C': [1,2,3,4,5,6,7,8,9.10,11],
              'penalty': ["l1", "l2", "none"]}
Model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(Model, param_grid, verbose=3)
grid

GridSearchCV(estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [1, 2, 3, 4, 5, 6, 7, 8, 9.1, 11],
                         'penalty': ['l1', 'l2', 'none']},
             verbose=3)

In [ ]:
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/usr/local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV 1/5] END ................................C=1, penalty=l1; total time=   9.3s


/usr/local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV 2/5] END ................................C=1, penalty=l1; total time=   9.9s


...

/usr/local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV 4/5] END ................................C=3, penalty=l1; total time=  12.7s


/usr/local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV 5/5] END ................................C=3, penalty=l1; total time=  10.6s
[CV 1/5] END ................................C=3, penalty=l2; total time=   0.2s
[CV 2/5] END ................................C=3, penalty=l2; total time=   0.2s
[CV 3/5] END ................................C=3, penalty=l2; total time=   0.2s
[CV 4/5] END ................................C=3, penalty=l2; total time=   0.2s
[CV 5/5] END ................................C=3, penalty=l2; total time=   0.2s
[CV 1/5] END ..............................C=3, penalty=none; total time=   0.0s
[CV 2/5] END ..............................C=3, penalty=none; total time=   0.0s
[CV 3/5] END ..............................C=3, penalty=none; total time=   0.0s
[CV 4/5] END ..............................C=3, penalty=none; total time=   0.0s
[CV 5/5] END ..............................C=3, penalty=none; total time=   0.0s


/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 454, in _check_solver
    raise ValueError(
ValueError: penalty='none' is not supported for the liblinear solver

  warnings.warn("Estimator fit failed. The score on this train-test"
/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-te

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

# Save the Model

In [18]:
# save fitted model to file
# In the specific case of scikit-learn, it may be better to use joblib’s replacement of
# pickle (dump & load), which is more efficient on objects that carry large numpy arrays
# internally as is often the case for fitted scikit-learn estimators, but can only pickle
# to the disk and not to a string:

import joblib
filename = 'five_hoods_LR.sav'
joblib.dump(grid, filename)

['five_hoods_LR.sav']

## Test

In [20]:
Model = joblib.load("five_hoods_LR.sav")
Model.predict(X_test_scaled)

array(['Buckhead', 'Downtown', 'Downtown', ..., 'Downtown', 'Downtown',
       'Midtown'], dtype=object)